In [120]:
import warnings
import numpy as np
import pandas as pd
import fsspec
import xarray as xr
from dask.diagnostics import ProgressBar
import matplotlib.pyplot as plt
import hvplot.xarray
import holoviews as hv

warnings.simplefilter('ignore') # filter some warning messages
xr.set_options(display_style="html")  #display dataset nicely

In [95]:
buoy = 'B8s30w'
pirata = xr.open_dataset(f'http://goosbrasil.org:8080/pirata/{buoy}.nc')
pirata = pirata.where((pirata!=pirata.missing_value)&(pirata!=-9999))

pirata

<xarray.Dataset>
Dimensions:        (depth_s: 8, depth_t: 15, time: 5761)
Coordinates:
  * depth_s        (depth_s) float32 1.0 5.0 10.0 20.0 40.0 60.0 80.0 120.0
  * depth_t        (depth_t) float32 1.0 5.0 10.0 13.0 ... 180.0 300.0 500.0
  * time           (time) datetime64[ns] 2005-08-20T12:00:00 ... 2021-05-28T1...
Data variables: (12/43)
    rad            (time) float32 nan 267.1 275.5 273.0 ... 244.6 248.8 238.3
    salinity_q     (time, depth_s) float32 nan nan nan nan ... nan nan 0.0 0.0
    rh_q           (time) float32 1.0 1.0 1.0 1.0 1.0 ... 2.0 2.0 2.0 2.0 2.0
    salinity_s     (time, depth_s) float32 nan nan nan nan ... nan nan 0.0 0.0
    qsen_q         (time) float32 2.0 2.0 2.0 2.0 2.0 ... nan nan nan nan nan
    rh_s           (time) float32 5.0 5.0 5.0 5.0 5.0 ... 1.0 1.0 1.0 1.0 1.0
    ...             ...
    salinity_id    (time, depth_s) float32 nan nan nan nan ... nan nan 0.0 nan
    rad_q          (time) float32 nan 1.0 1.0 1.0 1.0 ... 2.0 2.0 2.0 2.0 2.0
    longitude      (time) float32 329.5 329.5 329.5 329.5 ... 329.4 329.4 329.4
    salinity       (time, depth_s) float32 nan nan nan nan ... nan nan nan 0.0
    ws             (time) float32 7.7 6.2 7.1 8.2 7.7 ... 5.6 5.9 7.1 6.5 5.8
    temperature_q  (time, depth_t) float32 1.0 nan nan nan ... 2.0 2.0 2.0 2.0
Attributes:
    comment:          These data were obtained from Pacific Marine Environmen...
    source:           Moored buoy from PIRATA
    history:          Created Mon May 31 18:08:44 2021
    description:      Buoy 8s30w
    missing_value:    -99999
    data_management:  SALT - Sea & Limno Technology
    institution:      GOOS-Brasil

## Temperatura do Ar e do Mar

In [115]:
sst = pirata.sel(depth_t=0,method='nearest').temperature
airt = pirata.airt
dif = (sst-airt).rename('SST-AIRT [C]')
rain = pirata.rain

In [166]:
rolling = airt.rolling(time=30,center=True).mean()
(
    airt.hvplot(color='#d2302d')*rolling.hvplot(color='black')+
    (airt-rolling).rename('Air Temperature Anomaly [C]').hvplot()+
    (airt-rolling).rename('Air Temperature Anomaly [C]').hvplot.hist(ylabel='Number of days')   
).cols(1)

:Layout
   .Overlay.I   :Overlay
      .Curve.I  :Curve   [time]   (airt)
      .Curve.II :Curve   [time]   (airt)
   .Curve.I     :Curve   [time]   (Air Temperature Anomaly [C])
   .NdOverlay.I :NdOverlay   [Element]
      :Histogram   [Air Temperature Anomaly [C]]   (Air Temperature Anomaly [C]_count)

In [130]:
fig = (
    rain.hvplot()+
    sst.hvplot()*airt.hvplot()
).cols(1)
    
fig

:Layout
   .Curve.I   :Curve   [time]   (rain)
   .Overlay.I :Overlay
      .Curve.I  :Curve   [time]   (temperature)
      .Curve.II :Curve   [time]   (airt)

# Wind data

In [169]:
wd = pirata.wd.resample(time='3d').mean()
ws = pirata.ws.resample(time='3d').mean()

In [170]:
wd.hvplot.hist()

:NdOverlay   [Element]
   :Histogram   [wd]   (wd_count)

In [174]:
wd = wd.where(wd>wd.quantile(0.02))

In [175]:
(
    wd.hvplot(color='red')+
    ws.hvplot()
).cols(1)

:Layout
   .Curve.I  :Curve   [time]   (wd)
   .Curve.II :Curve   [time]   (ws)

In [89]:
fig = pirata.temperature.hvplot(ylim=(0,500),cmap='Spectral_r')
fig.opts(invert_yaxis=True)

:Image   [time,depth_t]   (temperature)

In [179]:
sal = pirata.salinity

fig = sal.where(sal>5).hvplot(cmap='Spectral_r')
fig.opts(invert_yaxis=True)

:Image   [time,depth_s]   (salinity)